### League of Legends: Spring 2020 Dataset

This datasets contains all games across differnt professional League of Legends 

In [209]:
#import packages
import pandas as pd
import requests
import json
import pickle
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import numpy as np
import pdb

In [293]:
#----Read competitve history data set for list of patches ----#
patches = pd.read_csv('../data/2020 spring match data OraclesElixir 2020-05-15.csv')
patches = list(set(patches['patch']))
patches = [str(x) for x in patches]

In [297]:
def reformat_patch_number(patch):
    patch_split = patch.split('.')
    patch_split = [str(int(x)) for x in patch_split]
    patch_output = patch_split[0] + '.' + patch_split[1] + '.1'
    return(patch_output)
    

In [300]:
patches = [reformat_patch_number(x) for x in patches]

In [19]:
#----- In case we need to use API ------#

#One time api key creation
file_name = '/Users/horacefung/Documents/keys/lol_api_key'
#key = {'key':''} #don't show key
#output = open(file_name,'wb')
#pickle.dump(key, output)
#output.close()

input_file = open(file_name,'rb')
api_key = pickle.load(input_file)['key']
input_file.close()

#api_root_url = "https://www.googleapis.com/youtube/v3/videos?"
#url_params = "part=snippet&type=video&chart=mostPopular"

#url = api_root_url + url_params + api_key
#data = requests.get(url).json()

In [309]:
#----- Pull champion list -----#
url = 'http://ddragon.leagueoflegends.com/cdn/' + patches[-1] + '/data/en_US/champion.json'
data = requests.get(url).json()
champion_list = list(data['data'].keys())

In [281]:
#----- Pull Relevant Champion Patch Data ---------#

def data_pull(patches, champion_list):
    



dataset = {}

for patch in patches:
    for champion in champion_list
    



data = requests.get(url).json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [229]:
#Get additional spell data
url = 'http://ddragon.leagueoflegends.com/cdn/10.13.1/data/en_US/champion/Ezreal.json'
data = requests.get(url).json()
data['data']['Ezreal'].keys()

dict_keys(['info', 'allytips', 'recommended', 'key', 'name', 'lore', 'stats', 'enemytips', 'image', 'title', 'partype', 'tags', 'passive', 'skins', 'blurb', 'spells', 'id'])

In [271]:
def extract_data(champion, champion_name, patch):
    
    #Split into base stats & spells
    stats = champion['stats']
    passive = champion['passive']
    spells = champion['spells']
    
    #Keywords
    hard_cc_list = ['airborne', 'charm', 'flee', 'taunt', 'sleep', 'stun', 'supression', 'suspension', 
                    'stasis', 'pull', 'knock']
    soft_cc_list = ['blind', 'cripple', 'disarm', 'ground', 'knockdown', 'nearsight', 'root', 'silence', 'slow']
    gap_closer_list = ['dash', 'blink', 'leap', 'launch', 'movementspeed', 'teleport']
    vision_list = ['vision', 'sight']
    protection_list = ['shield', 'heal']
    
    #initialize variables
    hard_cc_value = 0
    soft_cc_value = 0
    spells_average_range_value = 0
    gap_closer_value = 0
    protection_value = 0
    
    #Passive
    passive = passive['description']
    passive = passive.lower()
    passive = passive.replace('movement speed', 'movementspeed') #specific logic for move speed
    passive = [porter.stem(x) for x in word_tokenize(passive)]
    hard_cc = len(list(set(passive) & set(hard_cc_list))) #unique key words
    soft_cc = len(list(set(passive) & set(soft_cc_list))) #unique key words
    protection = len(list(set(passive) & set(protection_list)))
    gap_closer = len(list(set(passive) & set(gap_closer_list)))
    
    #Update talley with passive
    hard_cc_value = hard_cc_value + hard_cc
    soft_cc_value = soft_cc_value + soft_cc
    protection_value = protection_value + protection
    gap_closer_value = gap_closer_value + gap_closer
    
    
    #Four spells
    for i in range(4):
        
        #----Extract text information from tooltip
        tooltip = spells[i]['tooltip'].lower()
        tooltip = tooltip.replace('movement speed', 'movementspeed') #specific logic for move speed
        tooltip = [porter.stem(x) for x in word_tokenize(tooltip)]
        hard_cc = len(list(set(tooltip) & set(hard_cc_list))) #unique key words
        soft_cc = len(list(set(tooltip) & set(soft_cc_list))) #unique key words
        protection = len(list(set(tooltip) & set(protection_list)))
        gap_closer = len(list(set(tooltip) & set(gap_closer_list)))
        spells_range = np.mean(spells[i]['range'])
        
        #Update talley
        hard_cc_value = hard_cc_value + hard_cc
        soft_cc_value = soft_cc_value + soft_cc
        protection_value = protection_value + protection
        gap_closer_value = gap_closer_value + gap_closer
        spells_average_range_value = np.mean([spells_average_range_value, spells_range])
    
    #-----Setup Dataframe---------#
    dict_temp = {'champion' : champion_name,
                 'patch' : patch,
                 'hard_cc_value' : hard_cc_value, 
                 'soft_cc_value' : soft_cc_value,
                 'spells_average_range_value' : spells_average_range_value,
                 'gap_closer_value' : gap_closer_value,
                 'protection_value' : protection_value}
    
    dict_temp = {**stats, **dict_temp}
    
    output_df = pd.DataFrame(dict_temp, index = [0])
    output_df = output_df.set_index(['champion', 'patch'])
    
    
    return(output_df)

In [272]:
thresh_spells = data['data']['Ezreal']
extract_data(thresh_spells, 'Ezreal', '10.13.1')

,,armor,armorperlevel,attackdamage,attackdamageperlevel,attackrange,attackspeed,attackspeedperlevel,crit,critperlevel,gap_closer_value,...,movespeed,mp,mpperlevel,mpregen,mpregenperlevel,protection_value,soft_cc_value,spellblock,spellblockperlevel,spells_average_range_value
champion,patch,,,,,,,,,,,,,,,,,,,,,
Ezreal,10.13.1,22,3.5,60,2.5,550,0.625,2.5,0,0,1,...,325,375,50,8.5,0.65,0,0,30,0.5,12834.375


In [100]:
clean_list = [porter.stem(x) for x in word_tokenize(tooltip)]

In [239]:
'whatever OMG'.lower()

'whatever omg'

In [150]:
len(list(set(clean_list) & set(hard_cc)))

2

In [ ]:
#Notes

#1. Rolling average/window for recent performance for team e.g. Cloud 9
#2. Don't use player name, we should moneyball this and break down the player by their stats
# then do a stats coming into this game 